## Baseline 모델
 Baseline 모델은 일반적인 머신러닝 파이프라인의 모든 과정을 포함하는 가장 기초적인 모델이다.  
 ### Tabular 데이터를 다루는 캐글에서 머신러닝 파이프라인의 일반적인 순서는 다음과 같다.  
 1. 데이터 전처리   
 2. 피처 엔지니어링  
 3. 머신러닝 모델 학습  
 4. 테스트 데이터 예측 및 캐글 업로드
 
 #### 1. 데이터 전처리
 Baseline 모델 구축을 위하여 데이터 전처리 과정에서는 다음 작업을 수행한다.  
 1) 제품 변수의 결측값을 0으로 대체한다. 제품 보유 여부에 대한 정보가 없으면, 해당 제품을 보유하고 있지 않다고 가정한다.  
 2) 훈련 데이터와 테스트 데이터를 통합한다. 훈련 데이터와 테스트 데이터는 날짜 변수로 쉽게 구분이 가능하다. 동일한 24개의 고객변수를 공유하고 있으며, 테스트 데이터에 없는 23개의 제품 변수는 0으로 채운다.  
 3) 범주형, 수치형 데이터를 전처리한다. 범주형 데이터는 .factorize()를 통해 Label Encoding을 수행한다. 데이터 타입이 object로 표현되는 수치형 데이터에서는 .unique()를 통해 특이값들을 대체하거나 제거하고, 정수형 데이터로 변환한다.  
 4) 추후, 모델 학습에 사용할 변수 이름을 features 리스트에 미리 담는다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My Drive/Kaggle/Santander

/content/drive/My Drive/Kaggle/Santander


In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다.
trn = pd.read_csv('train_ver2.csv.zip')
tst = pd.read_csv('test_ver2.csv.zip')

## 데이터 전처리 ##

# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
  tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 
                    'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
  df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추가한다.
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

### 2. 피처 엔지니어링
피처 엔지니어링 단계에서는 머신러닝 모델 학습에 사용할 파생 변수를 생성한다.   
Baseline 모델에서는 전체 24개의 고객 변수와, 4개의 날짜 변수 기반 파생변수, 그리고 24개의 lag-1 변수를 사용한다.  
고객이 첫 계약을 맺은 날짜를 의미하는 fecha_alta와 고객이 마지막으로 1등급이었던 날짜를 의미하는 ult_fec_cli_1t 변수에서 각각 연도와 월 정보를 추출한다. 그 외에도 날짜 변수를 활용한 파생 변수는 다양하다. 예를 들어, 두 개의 날짜 변수 간의 차이값을 파생 변수로 생성한다거나, 졸업식이나 방학 등의 특별한 날짜까지의 거리를 수치형 변수로 생성할 수 있다.  
결측값은 임시로 -99로 대체한다. scikit-learn에서 제공하는 모델은 결측값을 입력값으로 받지 않고 실행 에러가 발생하지만, xgboost 모델에서는 결측값도 정상적인 입력값으로 받는다. 데이터가 결측되었다는 것도 하나의 정보로 인식하고 모델 학습에 활용되지만, 그냥 -99로 설정한다.  
시계열 데이터에서는 고객의 과거 데이터를 기반으로 다양한 파생 변수를 생성할 수 있다. 예를 들어, 고객의 나이가 최근 3개월 동안 변동이 있었는지(최근 3개월 안에 생일이었는지)를 이진 변수로 생성하거나, 한 달 전에 구매한 제품에 대한 정보를 변수로 사용할 수 있고, 최근 6개월 평균 월급을 계산할 수도 있다.  
santander 경진대회에서는 N개월 전에 금융 제품을 보유하고 있었는지 여부를 나타내는 lag변수가 좋은 파생 변수로 작용했다. 24개의 금융 제품 변수에 대하여 1개월 전, 2개월 전, 3개월 전 보유 여부를 현재 고객의 데이터로 활용하는 것이다. baseline 모델에서는 1개월 전 정보만을 가져다 사용하는 lag-1 변수를 사용한다. lag-5 변수까지 구현하여 성능 개선을 확인해 볼 것..  

In [0]:
# (피처 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값을 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# (피처 엔지니어링) lag-1 데이터를 생성한다.

# 날짜를 숫자로 변환하는 함수.
def date_to_int(str_date):
  Y, M, D = [int(a) for a in str_date.strip().split("-")]
  int_date = (int(Y)-2015)*12 + int(M)
  return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다.
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers' 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다.
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
  prev = prod + '_prev'
  df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

###
### Baseline 모델 이후, 다양한 피처 엔지니어링을 여기에 추가한다.
###

